# LangChain Expression Language (LCEL)

In [1]:
import os
# import openai

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file
# openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
#!pip install pydantic==1.10.8

In [11]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [12]:
prompt = ChatPromptTemplate.from_template(
    "{topic}에 대한 짧은 농담을 말해줘."
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [13]:
chain = prompt | model | output_parser

In [14]:
response = chain.invoke({"topic": "bears"})
print(response)

왜 곰들은 항상 춥죠? 

왜냐하면 그들은 언제나 겨울모드에 있거든요!


## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [17]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [23]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [24]:
results = retriever.invoke( "where did harrison work?")
print(results)

[Document(page_content='harrison worked at kensho'), Document(page_content='bears like to eat honey')]


In [25]:
results = retriever.invoke("what do bears like to eat?")
print(results)

[Document(page_content='bears like to eat honey'), Document(page_content='harrison worked at kensho')]


In [26]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [27]:
from langchain.schema.runnable import RunnableMap

In [28]:
chain = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [29]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [30]:
inputs = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
})

In [31]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind

and OpenAI Functions

In [32]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [33]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [34]:
runnable = prompt | model

In [35]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 64, 'total_tokens': 80}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-9f9a285c-36f9-43ed-9e48-6d41f1bcf27e-0', usage_metadata={'input_tokens': 64, 'output_tokens': 16, 'total_tokens': 80})

In [36]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [37]:
model = model.bind(functions=functions)

In [38]:
runnable = prompt | model

In [39]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"patriots"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 99, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-600262c2-ed8a-4adc-859c-3647da56f043-0', usage_metadata={'input_tokens': 99, 'output_tokens': 18, 'total_tokens': 117})

- 패트리어츠(Patriots)는 미국의 프로 미식축구 팀인 뉴잉글랜드 패트리어츠(New England Patriots)를 줄여서 부르는 말입니다. 이 팀은 NFL(National Football League)에 속해 있으며, 매사추세츠주 폭스버러에 기반을 두고 있습니다. 패트리어츠는 NFL에서 매우 성공적인 팀 중 하나로, 여러 차례 슈퍼볼 우승을 차지한 경력이 있습니다.

## Fallbacks

In [66]:
from langchain.llms import OpenAI # 옛날 모델 
import json

**Note**: Due to the deprecation of OpenAI's model `text-davinci-001` on 4 January 2024, you'll be using OpenAI's recommended replacement model `gpt-3.5-turbo-instruct` instead.

In [69]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="gpt-3.5-turbo-instruct"
)

simple_chain = simple_model | json.loads

In [70]:
challenge = "세 개의 시를 JSON 블롭으로 작성하세요. 각 시는 제목, 작가, 첫 줄로 구성된 JSON 블롭입니다."

In [71]:
response = simple_model.invoke(challenge)
print(response)



{
  "title": "봄",
  "author": "김소월",
  "first_line": "봄이 오면 산들도 꽃피고"
}

{
  "title": "가을",
  "author": "윤동주",
  "first_line": "가을이 오면 나는 더 사랑하리라"
}

{
  "title": "겨울",
  "author": "정지용",
  "first_line": "겨울이 오면 나는 더 추억하리라"
}


In [72]:
type(response)

str

**Note**: The next line is expected to fail.

In [74]:
from langchain_openai import ChatOpenAI # 최근 모델

model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [75]:
response = chain.invoke(challenge)

In [76]:
print(response)

[{'title': '봄', 'author': '윤동주', 'first_line': '봄이 오나 봄이 오나 봄이 오나'}, {'title': '가을', 'author': '김소월', 'first_line': '하늘 높이 구름 없이 맑은 가을하늘'}, {'title': '겨울', 'author': '이상', 'first_line': '눈 내리는 밤이면 눈 내리는 밤이면'}]


In [77]:
type(response[0])

dict

- fallbacks 를 쓰면 에러가 나도 원래 chain로 돌아가서 실행을 한다.

In [49]:
final_chain = simple_chain.with_fallbacks([chain])

In [50]:
response = final_chain.invoke(challenge)
print(response)

[{'title': '봄', 'author': '윤동주', 'first_line': '봄이 오나 봄이 오나 봄이 오나'}, {'title': '가을', 'author': '김소월', 'first_line': '하늘 높이 구름 없이 맑은 가을하늘'}, {'title': '겨울', 'author': '이상', 'first_line': '눈 내리는 밤이면 눈 내리는 밤이면'}]


## Interface

In [78]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [79]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes? \n\nBecause they have bear feet!"

In [80]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why did the bear break up with his girlfriend?\nBecause he couldn't bear the relationship any longer!",
 'Why are frogs so happy?\n\nBecause they eat whatever bugs them!']

In [81]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 have
 bear
 feet
!



In [82]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears like fast food? Because they can't catch it!"